In [13]:
from pipeline import *
import pandas as pd

In [14]:
year = 10

df_teams, df_teams_post, df_series_post, df_players, df_players_teams, df_coaches, df_awards_players = load_data()

df_teams_merged = global_merge(df_teams, df_teams_post, df_series_post,
                                   df_players, df_players_teams, df_coaches, df_awards_players, year)

df_teams_merged.head(20)

/home/sparks/Code/AC/funcs/statistical_analysis.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coaches['win_percentage_coach'] = (df_coaches['won'] + df_coaches['post_wins']) / (df_coaches['won'] +


,tmID,year,mean,confID,playoff,win_percentage_coach,awards
0,ATL,9,0.194587,EA,0,-2.235235,2
1,ATL,10,0.780557,EA,1,-2.235235,5
2,CHA,1,-0.000280,EA,0,-1.295938,2
3,CHA,2,0.469957,EA,1,0.533667,3
4,CHA,3,0.288790,EA,1,0.533667,3
5,CHA,4,0.561874,EA,1,-0.478402,3
6,CHA,5,0.321382,EA,0,-0.478402,2
7,CHA,6,0.384588,EA,0,0.000000,3
8,CHA,7,0.619900,EA,0,-0.857600,1
9,CHI,7,0.297826,EA,0,-2.026503,0
